### 第二部分

训练前的准备

In [2]:
import pandas as pd

# 读取数据
train = pd.read_csv("../dataset/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../dataset/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("../dataset/unlabeledTrainData.tsv", header=0, \
                              delimiter="\t", quoting=3)
# 验证数据
print(f"Read {train['review'].size} labeled train reviews, \
      {test['review'].size} labeled test reviews, \
      and {unlabeled_train['review'].size} unlabeled train reviews.")

Read 25000 labeled train reviews,       25000 labeled test reviews,       and 50000 unlabeled train reviews.


In [3]:
# 类似part1, 我们需要清理数据
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

def review_to_wordlist( review, remove_stopwords=False ):
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 2. Remove non-letters&number
    # 教程说最好连数字也不要删除， 只需要稍微修改正则表达式的匹配规则即可
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 5. Return a list of words
    return words
s = "We are good friend 111 234."
print(review_to_wordlist(s))

d:\ANACONDA\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


['we', 'are', 'good', 'friend', '111', '234']


d:\ANACONDA\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
d:\ANACONDA\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).ep

Word2Vec的需要的输入格式是以list为单位的句子, 我们采用nltk中的punkt来分割句子.请务必区分list.append()和 += 的区别

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# 下载完后注释掉
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [5]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
d:\ANACONDA\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


d:\ANACONDA\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
d:\ANACONDA\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserW

In [6]:
# 查看一下处理后的结果
print(len(sentences))
print(sentences[0])
print(sentences[1])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


和part1的结果相比, part2的结果中含有stopwords, 并保留有相对顺序说明words2vec在分析时考虑了上下文

确定参数并训练模型

In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2025-04-14 17:10:20,991 : INFO : collecting all words and their counts
2025-04-14 17:10:20,992 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-04-14 17:10:21,027 : INFO : PROGRESS: at sentence #10000, processed 227240 words, keeping 18038 word types
2025-04-14 17:10:21,060 : INFO : PROGRESS: at sentence #20000, processed 454577 words, keeping 25324 word types
2025-04-14 17:10:21,095 : INFO : PROGRESS: at sentence #30000, processed 675275 words, keeping 30478 word types
2025-04-14 17:10:21,131 : INFO : PROGRESS: at sentence #40000, processed 903015 words, keeping 34863 word types


Training model...


2025-04-14 17:10:21,166 : INFO : PROGRESS: at sentence #50000, processed 1123504 words, keeping 38329 word types
2025-04-14 17:10:21,205 : INFO : PROGRESS: at sentence #60000, processed 1346265 words, keeping 41338 word types
2025-04-14 17:10:21,241 : INFO : PROGRESS: at sentence #70000, processed 1570739 words, keeping 43986 word types
2025-04-14 17:10:21,279 : INFO : PROGRESS: at sentence #80000, processed 1791249 words, keeping 46400 word types
2025-04-14 17:10:21,316 : INFO : PROGRESS: at sentence #90000, processed 2016723 words, keeping 48869 word types
2025-04-14 17:10:21,354 : INFO : PROGRESS: at sentence #100000, processed 2239896 words, keeping 50980 word types
2025-04-14 17:10:21,388 : INFO : PROGRESS: at sentence #110000, processed 2460901 words, keeping 52890 word types
2025-04-14 17:10:21,426 : INFO : PROGRESS: at sentence #120000, processed 2684304 words, keeping 54967 word types
2025-04-14 17:10:21,465 : INFO : PROGRESS: at sentence #130000, processed 2911246 words, keep

测试我们的模型

In [ ]:
# 返回最不相识的
# 新版的gensim将词向量相关内容移动到.wv中
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [ ]:
# 我们的模型对细微的差异很敏感
# 如: 能够区分城市和国家
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [14]:
model.wv.most_similar("awful")

[('terrible', 0.7552964687347412),
 ('atrocious', 0.7382639050483704),
 ('horrible', 0.7245572805404663),
 ('abysmal', 0.7207901477813721),
 ('dreadful', 0.7166836857795715),
 ('appalling', 0.7005763649940491),
 ('horrid', 0.6964068412780762),
 ('horrendous', 0.6856689453125),
 ('amateurish', 0.6398955583572388),
 ('laughable', 0.6197744607925415)]

### 第三部分

在这一部分, 我们尝试如何将训练好的词向量运用到监督学习中.

word2vec的词向量保存在wv.vectors中.其中行数为字典中的word number, 列数为训练模型时规定的num_feature

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

print(type(model.wv.vectors))
print(model.wv.vectors.shape)

<class 'numpy.ndarray'>
(16731, 300)


可以查看特定单词的词向量

In [9]:
print(model.wv["flower"])
print(len(model.wv["flower"]))

[ 0.01041056  0.04125478  0.02076882  0.08313817 -0.02562958 -0.08931556
  0.0251859   0.1030851   0.07478915 -0.07058468  0.05776194  0.03171209
  0.01501744  0.09055397 -0.04950409 -0.05708684  0.04555706 -0.11122721
 -0.00563609 -0.0472794  -0.02630693 -0.00483534  0.007275    0.07415794
  0.06638899 -0.09303596 -0.05580951  0.0193401  -0.08157014 -0.00214469
  0.0970979  -0.01206447 -0.03748423 -0.03147227 -0.01008049  0.05060023
 -0.00662906 -0.02343504  0.03069458 -0.01843819 -0.02163189  0.04567805
  0.10800102 -0.04625378 -0.06781906  0.04858699  0.04128491 -0.02616478
  0.01666732  0.06230603  0.07332691 -0.02478444 -0.02854712 -0.0251361
 -0.12528984  0.0294585   0.08240269  0.00387076  0.00197295 -0.00840849
 -0.09343222  0.08877736 -0.02080078  0.09056281 -0.08471962  0.10664986
 -0.05752569  0.04569745 -0.02267064 -0.01436201 -0.0004109  -0.0133284
  0.05110568  0.02125193  0.01463221 -0.08360124  0.00918045  0.1312618
 -0.07167825  0.18238837  0.01082698  0.01080399 -0.00

### 从单词到段落: 尝试1,向量平均

我们可以将整个review的词向量简单的相加求平均, 用平均后向量来代表整个段落.

In [ ]:
import numpy as np

# 将review转换为word vector
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed # index2word 
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model.wv[word])
    
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    reviewFeatureVecs = np.zeros((len(reviews),num_features), dtype="float32")
    for i, review in enumerate(reviews):
        if i % 1000 == 0:
            print("Review %d of %d".format(i, len(reviews)))
        reviewFeatureVecs[i] = makeFeatureVec(review, model, num_features)
    return reviewFeatureVecs

In [ ]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

print("Creating average feature vecs for train reviews")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )